# Rubin (DP0) & Roman (Troxel+23) images - alternate.

Contact: Melissa Graham, Andrés A. Plazas Malagón

**Based on a tutorial by Chien-Hao Lin.** 

**This tutorial is similar to the "3_images_tutorial_DP0+Roman.ipynb" tutorial in this directory, except it uses Firefly for image display and incorporates steps from the DP0.2 ["NB05 Source Detection and Measurement"](https://dp0-2.lsst.io/_static/nb_html/DP02_05_Source_Detection_and_Measurement.html) tutorial for image processing of the Roman simulations.**

Date: Mon Nov 14 2024

RSP Image: Weekly 2024_42

Goal: Recreate Chien-Hao's notebook and use Roman images to help with deblending Rubin images.

## Introduction

Space-based images have much higher resolution.
Stars and galaxies that are very close together or even overlapping (blended) due to chance
alignments along the line-of-sight can be better distinguished in higher resolution images.
So can actual galaxy mergers in close physical proximity, though this is less common.

It is possible to use the locations of physically distinct objects (deblended objects)
from higher resolution images to make more accurate photometry measurements.
In cases where the higher and lower resolution images are obtained in the same filters,
and with similar depths, it makes sense just to use the higher resolution images alone.

However, Rubin will obtain data in optical filters and Roman in infrared filters.
In this case, using the higher-resolution infrared images to determine the number and
location of distinct objects, and then make photometric measurements in Rubin's
optical-range images, can improve the optical photometric measurements.

Roman DC2 Simulated Images and Catalogs at IRSA IPAC:<br>
https://irsa.ipac.caltech.edu/data/theory/Roman/Troxel2023/overview.html

Troxel et al. (2023):<br>
https://academic.oup.com/mnras/article/522/2/2801/7076879?login=false

## Set up

In [ ]:
import numpy as np

import lsst.geom
import lsst.afw.image as afwImage
import lsst.afw.display as afwDisplay
from lsst.daf.butler import Butler as dafButler

import lsst.daf.base as dafBase
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask
from lsst.meas.algorithms.detection import SourceDetectionTask
from lsst.meas.deblender import SourceDeblendTask
from lsst.meas.base import SingleFrameMeasurementTask
import lsst.afw.table as afwTable

import lsst.meas.algorithms as measAlg
import lsst.afw.math as afwMath
import lsst.afw.image as afwImage

Set the display backed to be Firefly.

In [ ]:
afwDisplay.setDefaultBackend('firefly')

Instantiate the butler.

In [ ]:
butler = dafButler('dp02', collections='2.2i/runs/DP0.2')

Define `ra` and `dec`, the central coordinates of interest.

Define the scale, in arcseconds per pixel, of Rubin and Roman images.

Define the stamp size to use when visualizing the images (i.e., the cutout size), in Rubin pixels; then use `stampsize / scale_ratio` as the extent when visualizing Roman images.

In [ ]:
ra, dec = 54.28, -38.30
rubin_scale = 0.2
roman_scale = 0.0575
stampsize = 150
scale_ratio = rubin_scale/roman_scale

Convert the coordinates to type `SpherePoint`.

In [ ]:
radec = lsst.geom.SpherePoint(ra, dec, lsst.geom.degrees)

For Rubin images: identify Rubin DP0.2 butler tract and patch for the desired coordinates.

In [ ]:
skymap = butler.get('skyMap')
tract = skymap.findTract(radec).tract_id
patch = skymap.findTract(radec).findPatch(radec).getSequentialIndex()
print(tract, patch)

For Roman image and Point Spread Function (PSF) files: four filters for one patch of deeply coadded Roman images have been stored in the shared space in the `/project` directory.

In [ ]:
ro_img_path = '/project/melissagraham2/troxel2023/'
ro_img_fnms = {
    'Y106': 'dc2_Y106_54.24_-38.3.fits',
    'J129': 'dc2_J129_54.24_-38.3.fits',
    'H158': 'dc2_H158_54.24_-38.3.fits',
    'F184': 'dc2_F184_54.24_-38.3.fits'
}


ro_psf_path = '/project/plazas/troxel2023/psf/coadd/'
ro_psf_fnms = {
    'Y106': 'dc2_Y106_54.24_-38.3_psf.fits',
    'J129': 'dc2_J129_54.24_-38.3_psf.fits',
    'H158': 'dc2_H158_54.24_-38.3_psf.fits',
    'F184': 'dc2_F184_54.24_-38.3_psf.fits'
}

## Visualize images

### Rubin images

For each filter, retrieve the `deepCoadd` patch and display the cutout in Firefly frames 1 through 6.

In [ ]:
lsst_bands = ['u', 'g', 'r', 'i', 'z', 'y']
for i in range(6):
    print(i+1, lsst_bands[i])
    afw_display = afwDisplay.Display(frame=i+1)
    dataId = {'tract': tract, 'patch': patch, 'band': lsst_bands[i]}
    image = butler.get('deepCoadd', dataId=dataId)
    size = stampsize
    extent = lsst.geom.ExtentI(size, size)
    cutout = image.getCutout(radec, extent)
    afw_display.mtv(cutout)
    afw_display.setMaskTransparency(100)
    del dataId, image, size, extent, cutout

### Roman images

Load and display a small cutout from each of the four images in Firefly frames 7 through 10.

> **Warnings:** Below, the warnings about unreadable mask extensions can be disregarded for the purposes of this tutorial, but generally when using the LSST Science Pipelines with non-Rubin data, all warnings should be followed up and third-party data sets might need to be reformatted to work properly.
In this case the images have four extensions: SCI, WHT, CTX, ERR.
But the `readFits` function expects MASK and IMAGE extensions.

In [ ]:
roman_bands = ['Y106', 'J129', 'H158', 'F184']
for i, band in enumerate(roman_bands):
    afw_display = afwDisplay.Display(frame=i+7)
    fnm = ro_img_path + ro_img_fnms[band]
    print(i+7, fnm)
    image = afwImage.ExposureF.readFits(fnm)
    size = stampsize * scale_ratio
    extent = lsst.geom.ExtentI(size, size)
    cutout = image.getCutout(radec, extent)
    afw_display.mtv(cutout)
    del fnm, image, size, extent, cutout

### Clean up the Firefly window

Close each panel by clicking on the 'X' in the upper right corner until the display again says "Firefly Ready" and "Awaiting Python API Commands".

## Visualize detected sources

### Rubin

Source detection has already been run on the Rubin DP0 images.

The threshold is 5-sigma.

Display the r-band image cutout and overplot detected sources.

In [ ]:
dataId = {'tract': tract, 'patch': patch, 'band': 'r'}

In [ ]:
objects = butler.get('objectTable', dataId=dataId)

In [ ]:
# objects

Extract coordinate data into `numpy` arrays.

In [ ]:
ra_vals = np.asarray(objects['coord_ra'], dtype='float')
dec_vals = np.asarray(objects['coord_dec'], dtype='float')
x_vals = np.asarray(objects['x'], dtype='float')
y_vals = np.asarray(objects['y'], dtype='float')

Display the r-band cutout.

In [ ]:
afw_display = afwDisplay.Display(frame=1)
image = butler.get('deepCoadd', dataId=dataId)
size = stampsize
extent = lsst.geom.ExtentI(size, size)
cutout = image.getCutout(radec, extent)
afw_display.mtv(cutout)
afw_display.setMaskTransparency(100)

Mark detected objects with orange circles.

The stampsize of 150 x 150 pixels is 30 x 30 arcsec, or about 0.008 x 0.008 degrees.
Only plot objects thata re in the cutout.

In [ ]:
tx = np.where((ra_vals > ra - 0.01) &
              (ra_vals < ra + 0.01) &
              (dec_vals > dec - 0.01) &
              (dec_vals < dec + 0.01))[0]
print(len(tx))

In [ ]:
with afw_display.Buffering():
    for x in tx:
        afw_display.dot('o', x_vals[x], y_vals[x],
                        size=5, ctype='orange')

Clean up.

In [ ]:
del ra_vals, dec_vals, x_vals, y_vals
del tx
del dataId, image, size, extent, cutout

### Roman


#### Use DP0.2 tutorial ["NB05 Source Detection and Measurement"](https://dp0-2.lsst.io/_static/nb_html/DP02_05_Source_Detection_and_Measurement.html) as a guide
Start by setting up the configurations for each task.

In [ ]:
config_characterize = CharacterizeImageTask.ConfigClass()
config_characterize.psfIterations = 2

config_detection = SourceDetectionTask.ConfigClass()
config_detection.thresholdValue = 5
config_detection.thresholdType = "stdev"

config_deblend = SourceDeblendTask.ConfigClass()
config_deblend.propagateAllPeaks = True
config_deblend.maskPlanes=[]

config_measure = SingleFrameMeasurementTask.ConfigClass() 

Define a minimal schema and add a few extra fields to avoind warnings.

In [ ]:
schema = afwTable.SourceTable.makeMinimalSchema()
raerr = schema.addField("coord_raErr", type="F")
decerr = schema.addField("coord_decErr", type="F")

algMetadata = dafBase.PropertyList()

Define the image processing tasks. 

In [ ]:
detectionTask = SourceDetectionTask(schema=schema, config=config_detection)
deblendTask = SourceDeblendTask(schema=schema, config=config_deblend)
measureTask = SingleFrameMeasurementTask(schema=schema,
                                         config=config_measure,
                                         algMetadata=algMetadata)

Get the Roman image and PSF.

In [ ]:
fnm = ro_img_path + ro_img_fnms['H158']
fnm_psf = ro_psf_path + ro_psf_fnms['H158']
print(fnm)
print (fnm_psf)

Set the PSF in the full image, and make a cutout.

In [ ]:
image = afwImage.ExposureF.readFits(fnm)
psf = measAlg.KernelPsf(afwMath.FixedKernel(afwImage.ImageD(fnm_psf)))

image.setPsf(psf)

size = stampsize * scale_ratio
extent = lsst.geom.ExtentI(size, size)
cutout = image.getCutout(radec, extent)

In [ ]:
tab = afwTable.SourceTable.make(schema)

Run the detection, deblensing, and measurement tasks.

In [ ]:
result = detectionTask.run(tab, cutout)

In [ ]:
result.numPosPeaks

In [ ]:
sources = result.sources

In [ ]:
# sources

In [ ]:
deblendTask.run(cutout, sources)

In [ ]:
measureTask.run(measCat=sources, exposure=cutout)

In [ ]:
sources = sources.copy(True)

Display the Roman H158-band cutout in frame 2.

In [ ]:
afw_display = afwDisplay.Display(frame=2)
afw_display.mtv(cutout)
afw_display.setMaskTransparency(100)

Overplot detected sources.


In [ ]:
x_vals = np.asarray(sources.getX(), dtype='float')
y_vals = np.asarray(sources.getY(), dtype='float')

In [ ]:
with afw_display.Buffering():
    for i in np.arange(len(x_vals)):
        afw_display.dot('o', x_vals[i], y_vals[i],
                        size=20, ctype='orange')

Clean up.

In [ ]:
del x_vals, y_vals
del fnm, image, size, cutout, extent
del tab, result, sources

In [ ]:
del config_detection, config_deblend, config_measure
del algMetadata, schema
del detectionTask, deblendTask, measureTask